In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

def create_sample_images():
    """
    Hàm tạo ra 3 loại ảnh giả lập để kiểm thử:
    1. Ảnh nhiều nhiễu (Noisy)
    2. Ảnh tương phản thấp (Low Contrast)
    3. Ảnh nhiều chi tiết (High Detail)
    """
    h, w = 300, 300
    
    # --- 1. Tạo Ảnh Nhiều Nhiễu ---
    # Tạo nền đen có hình tròn trắng
    img_noisy = np.zeros((h, w), dtype=np.uint8)
    cv2.circle(img_noisy, (150, 150), 80, 200, -1)
    # Thêm nhiễu Gaussian (Gaussian Noise)
    noise = np.random.normal(0, 50, img_noisy.shape).astype(np.uint8)
    img_noisy = cv2.add(img_noisy, noise)

    # --- 2. Tạo Ảnh Tương Phản Thấp ---
    # Nền xám (100) và hình tròn xám hơn một chút (130) -> Chênh lệch ít
    img_low_contrast = np.full((h, w), 100, dtype=np.uint8)
    cv2.circle(img_low_contrast, (150, 150), 80, 130, -1)

    # --- 3. Tạo Ảnh Nhiều Chi Tiết ---
    # Tạo bàn cờ (Checkerboard) dày đặc
    img_detail = np.zeros((h, w), dtype=np.uint8)
    step = 10 # Kích thước ô cờ nhỏ
    for y in range(0, h, step):
        for x in range(0, w, step):
            if (x // step + y // step) % 2 == 1:
                img_detail[y:y+step, x:x+step] = 255

    return [
        ("Ảnh Nhiễu (Noisy)", img_noisy),
        ("Tương Phản Thấp (Low Contrast)", img_low_contrast),
        ("Nhiều Chi Tiết (High Detail)", img_detail)
    ]

def auto_canny(image, sigma=0.33):
    """
    Hàm tự động tính toán ngưỡng (threshold) dựa trên trung vị (median) của ảnh.
    Giúp Canny thích ứng tốt hơn với độ sáng khác nhau.
    """
    v = np.median(image)
    lower = int(max(0, (1.0 - sigma) * v))
    upper = int(min(255, (1.0 + sigma) * v))
    return cv2.Canny(image, lower, upper)

# --- CHƯƠNG TRÌNH CHÍNH ---

# 1. Lấy dữ liệu ảnh
dataset = create_sample_images()

# Thiết lập biểu đồ Matplotlib
fig, axes = plt.subplots(len(dataset), 3, figsize=(15, 12))
plt.subplots_adjust(hspace=0.4, wspace=0.3)

print("--- BẮT ĐẦU XỬ LÝ ẢNH ---")

# 2. Vòng lặp xử lý từng ảnh
for i, (title, original_img) in enumerate(dataset):
    print(f"Đang xử lý: {title}...")
    
    # Bước 1: Tiền xử lý (Làm mờ - Gaussian Blur)
    # Rất quan trọng để loại bỏ nhiễu trước khi tìm biên
    # Kernel (5,5) là mức trung bình, (7,7) hoặc (9,9) dùng cho ảnh rất nhiễu
    blurred_img = cv2.GaussianBlur(original_img, (5, 5), 0)
    
    # Bước 2: Áp dụng Canny
    # Cách 1: Dùng ngưỡng cố định (Thường không tối ưu cho mọi ảnh)
    # edges = cv2.Canny(blurred_img, 100, 200) 
    
    # Cách 2: Dùng ngưỡng động (Tùy chỉnh riêng cho từng loại để thấy rõ vấn đề)
    if "Nhiễu" in title:
        # Với ảnh nhiễu, cần ngưỡng cao để lọc bớt nhiễu còn sót
        lower, upper = 100, 200 
    elif "Thấp" in title:
        # Với ảnh tương phản thấp, biên rất mờ -> Cần ngưỡng cực thấp
        lower, upper = 20, 60 
    else:
        # Ảnh chi tiết
        lower, upper = 50, 150
        
    edges = cv2.Canny(blurred_img, lower, upper)

    # Bước 3: Hiển thị lên Matplotlib
    # Cột 1: Ảnh gốc
    axes[i, 0].imshow(original_img, cmap='gray')
    axes[i, 0].set_title(f"Gốc: {title}")
    axes[i, 0].axis('off')
    
    # Cột 2: Ảnh sau khi Blur (Xem hiệu quả khử nhiễu)
    axes[i, 1].imshow(blurred_img, cmap='gray')
    axes[i, 1].set_title("Sau Gaussian Blur")
    axes[i, 1].axis('off')

    # Cột 3: Kết quả Canny
    axes[i, 2].imshow(edges, cmap='gray')
    axes[i, 2].set_title(f"Canny Edges\n(Min:{lower}, Max:{upper})")
    axes[i, 2].axis('off')

print("--- HOÀN THÀNH ---")
plt.suptitle("Đánh giá thuật toán Canny trên các điều kiện ảnh khác nhau", fontsize=16)
plt.show()